# API and Web Scraping

# Web Scraping Air Quality Data

The QUALAR (https://qualar.apambiente.pt/) is a web platform of APA, the Portuguese Environment Agency, that displays online air quality data sampled by on air monitoring stations in Portugal. Unfortunately, the platform does not expose to the final user, or provides documentation how to use the API service that was implemented for web users downloads. Downloads are generated as XLSX files.

However, it is possible to hack the source code of the webpage to identify that there is an implemented API, and that it can be used to facilite efficient download of data. This exercise will demonstrate that, with the following steps:

* check how are downloads generated from the website
* identify and collect the parameters that define the data download
* use the API to download data
* as a bonus, visualise a timeseries of a data quality variable.

# About the Data

Visit the data download page of QUALAR, at https://qualar.apambiente.pt/downloads. It displays a table with a list of Air Monitoring Stations, with the following columns:

* Region (Região)
* Municipality (Concelho)
* Station (Estação)
* Station type (Tipo de Estação), with categories traffic, industrial and background
* Área type (Tipo de Área), with categories urban, suburban and rural
* columns for the following pollutants: O3, NO2, CO, SO2, PM10, PM2.5, C6H6, other

# 1. Verify how are downloads generated by inspecting the webpage source code

It is possible to inspect the source code of the table, and the behaviour of the page when a download is solicited. Checking this we can try to identify which methods are used to provide data to users. If we manage to verify that the web page is served by an API, and we can identify which parameters define a request, then it would be possible to generate a script to speed up downloads.

## 1.1. Activate the Inspect Tool of the source code of the webpage.

Open in your web browser, navigate to https://qualar.apambiente.pt/downloads.Afterwards, in the menu of your browser, find the option Web Developer Tools* or Developer Tools (in Firefox or Chrome, you will find it in More tools). This will open a new panel in the browser.*

## 1.2. Check the method to generate downloads

As mentioned before, clicking on the name of a station will generate a download with all data for that station. This means that through the HTTP protocol, a request is made through the network. Checking which request was made (which URL request was send) is a good way of verifying what was the information send to the web server.

On the Developer Panel, click on the tab Netwotk. After that, click on the name of a station to make a download request. This will generate a new row on the panel, with the information about a GET* request.*

In this case we have the following Request URL:

We can identify the following sections in the URL:

* Host URL: https://qualar.apambiente.pt/api/download.php

Parameters:poluente_id=0&estacao_id=3082&data_inicio=2021-01-01&data_fim=2021-12-31&influencias=1,2,3&ambientes=1,2,4

The meaning of the parameters is more or less obvious:

* poluente_id - the ID of the pollutant. The value zero should mean all pollutants

* estacao_ID - the ID of the station

* data_inicio - starting date

* data_fim - ending date

* influencias - station type

* ambientes - area type

## 1.3. Verify that the method for download works

We have just identified an API service for downloading data. We can check if it works, testing with different parameters and see if results correspond to what is expected:

To download data only for year 2020, try the following modified URL:

https://qualar.apambiente.pt/api/download.php?poluente_id=0&estacao_id=3082&data_inicio=2020-01-01&data_fim=2020-12-31&influencias=1,2,3&ambientes=1,2,4

The challenge now is to discover the values of the IDs of the air monitoring stations (the parameter estacao_ID). If we find these, we can make a script to make automatic requests to download the data files.

## 1.4. Inspect the HTML source code

On the top left bar of the Developer tools panel, there is a arrow cursor option. Select this, and the place the mouse pointed on the name of one air monitoring station, in the table. You will verify that for each section of the web page where you hover your mouse, the corresponding HTML source code will be highlighted in the developer panel.

## 1.5. Select the section of the HTML code with the cell of the station name

# 2. Scrap the HTML source code to obtain IDs of the stations

We will scrap the html of table in https://qualar.apambiente.pt/downloads to obtain the IDs of the air monitoring stations. For that, we will use the phyton module BeautifulSoup.

## 2.1. Get the table content

In the web page, the table content is loaded dynamically. It is possible to use python libraries to obtain that html code, but it is easier to do it manually.

* go to the Inspector or Elements panel of the Developer Tool in your browser, depending of which browser you use
* navigate down through the html code hierarchy until you find the html tag <tbody>. Verify in the web page that the correct section of the table containing data is highlighted.
* right-click on top of the <tbody> tag, and select Copy --> Copy Outer HTML
* paste the memory context to a new file in your text editor (VS Code, Notepad++, etc), and save it with the name qualar_table.html in the raw-data o your project.

## 2.2. Import BeautifulSoup

In [1]:
from bs4 import BeautifulSoup

## 2.3. Get air pollution station IDs

In [2]:
# open the html file
with open("./raw-data/qualar_table.html", encoding="utf-8") as html_table:
    soup = BeautifulSoup(html_table, 'html.parser')

# create a list to store ids
station_ids = []


# get ids from html
for row in soup.find_all('label'):
    row_text = row['onclick']
    ids = row_text[row_text.find( '(' )+1:row_text.find( ')' )]
    station_ids.append(ids)
    
print(station_ids)

['4006', '5008', '5012', '3083', '3082', '3084', '2017', '3055', '3095', '3103', '1041', '1042', '3104', '3096', '2006', '2016', '1054', '2004', '5007', '6004', '6005', '2020', '1053', '1046', '7001', '2018', '2019', '3070', '3071', '3072', '3075', '3087', '3100', '3085', '3102', '1100', '1024', '1031', '1021', '1025', '1030', '1034', '1055', '2022', '3097', '3091', '1044', '3099', '1043', '5011', '1028', '1050', '6008', '4002', '4003', '4004', '1052', '3063', '3092', '3093', '3094', '4001', '3089', '1023', '1047', '1051', '3101', '1048', '2021']


# 4. Download files using the API

In [3]:
import requests
import datetime
import pandas as pd
from time import sleep
import urllib

__2010__

In [5]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2010-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2010-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2010/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2010-01-01&data_fim=2010-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2011__

In [4]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2011-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2011-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2011/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2011-01-01&data_fim=2011-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2012__

In [5]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2012-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2012-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2012/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2012-01-01&data_fim=2012-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2013__

In [6]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2013-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2013-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2013/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2013-01-01&data_fim=2013-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2014__

In [7]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2014-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2014-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2014/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2014-01-01&data_fim=2014-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2015__

In [8]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2015-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2015-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2015/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2015-01-01&data_fim=2015-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2016__

In [9]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2016-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2016-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2016/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2016-01-01&data_fim=2016-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2017__

In [10]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2017-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2017-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2017/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2017-01-01&data_fim=2017-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2018__

In [11]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2018-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2018-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2018/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2018-01-01&data_fim=2018-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2019__

In [12]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2019-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2019-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2019/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2019-01-01&data_fim=2019-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2020__

In [13]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2020-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2020-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2020/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2020-01-01&data_fim=2020-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

__2021__

In [14]:
# Define base URL of the service
base_url = 'https://qualar.apambiente.pt/api/download.php'

# Define params for the API request
params = {"poluente_id":"0", "influencias":"1,2,3", "ambientes":"1,2,4"}

# Define begining of the data period. You can change this for another year
params["data_inicio"] = "2021-01-01"

# Define end of the data period. You can change this for another year
params["data_fim"] = "2021-12-31"

# As this is not in production, we will replace the full list of ids by a smaller list of ids, to save time. 
# If you want all data, comment the next line
# station_ids = ['3095', '3103', '3104', '3096', '3070']

# This iteration will go through the list of IDs, request the excel file based on the URL and save it locally.
for id_value in station_ids:
    # define name of file to be saved
    filename = './qualar-data-2021/station_'+ id_value + '.xlsx'
    # add station_id to the parameters of the URL
    params["estacao_id"] = id_value
    try:
        r = requests.get(base_url, params)
    except requests.exceptions.RequestException as e: 
        raise SystemExit(e)
    print(r.url)
    with open(filename, 'wb') as f:
        f.write(r.content)

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=4006
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=5008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=5012
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=3083
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=3082
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=3084
https://qualar.apambiente.pt/api/d

https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=1050
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=6008
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=4002
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=4003
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=4004
https://qualar.apambiente.pt/api/download.php?poluente_id=0&influencias=1%2C2%2C3&ambientes=1%2C2%2C4&data_inicio=2021-01-01&data_fim=2021-12-31&estacao_id=1052
https://qualar.apambiente.pt/api/d

In [8]:
import os

# Let´s create a DataFrame

## 2010

In [13]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2010")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [14]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2010/"+file
    #read to a pandas dataframe
    df_2010 = pd.read_excel(xls)

In [15]:
# Let´s rename the column "Fail" to "Date"
df_2010.rename(columns={"Faial":"Date"}, inplace=True)
df_2010

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2010-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2010-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN
2,2010-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN
3,2010-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN
4,2010-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8755,2010-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN
8756,2010-12-31 20:00:00,NaN,NaN,NaN,NaN,NaN
8757,2010-12-31 21:00:00,NaN,NaN,NaN,NaN,NaN
8758,2010-12-31 22:00:00,NaN,NaN,NaN,NaN,NaN


In [16]:
df_2010.isnull().sum()

Date                              0
Dióxido de Enxofre (µg/m3)     1465
Partículas < 10 µm (µg/m3)     1134
Ozono (µg/m3)                   974
Dióxido de Azoto (µg/m3)       2574
Partículas < 2.5 µm (µg/m3)    1327
dtype: int64

In [17]:
df_2010.to_excel("air_quality_2010.xlsx")

## 2011

In [18]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2011")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [19]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2011/"+file
    #read to a pandas dataframe
    df_2011 = pd.read_excel(xls)

In [20]:
# Let´s rename the column "Fail" to "Date"
df_2011.rename(columns={"Faial":"Date"}, inplace=True)
df_2011

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2011-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN
2,2011-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN
3,2011-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN
4,2011-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8755,2011-12-31 19:00:00,1.3,3.2,70.6,1.1,3.0
8756,2011-12-31 20:00:00,1.3,4.0,72.6,1.0,2.7
8757,2011-12-31 21:00:00,1.2,4.0,71.8,1.0,2.0
8758,2011-12-31 22:00:00,1.4,3.7,66.2,1.1,2.0


## 2012

In [21]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2012")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [22]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2012/"+file
    #read to a pandas dataframe
    df_2012 = pd.read_excel(xls)

In [23]:
# Let´s rename the column "Fail" to "Date"
df_2012.rename(columns={"Faial":"Date"}, inplace=True)
df_2012

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2012-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2012-01-01 01:00:00,1.4,3.0,69.9,1.1,1.0
2,2012-01-01 02:00:00,1.2,3.0,73.0,1.1,1.5
3,2012-01-01 03:00:00,1.2,3.0,72.6,1.3,2.0
4,2012-01-01 04:00:00,1.5,3.0,74.9,1.2,2.0
...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,1.3,1.2,75.5,1.7,2.0
8780,2012-12-31 20:00:00,1.5,4.0,71.3,1.6,5.0
8781,2012-12-31 21:00:00,1.5,4.0,69.9,1.6,5.0
8782,2012-12-31 22:00:00,1.4,4.0,70.0,1.6,4.7


## 2013

In [24]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2013")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [25]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2013/"+file
    #read to a pandas dataframe
    df_2013 = pd.read_excel(xls)

In [26]:
# Let´s rename the column "Fail" to "Date"
df_2013.rename(columns={"Faial":"Date"}, inplace=True)
df_2013

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2013-01-01 00:00:00,1.5,3.0,72.4,3.1,3.0
1,2013-01-01 01:00:00,1.4,2.7,71.9,4.1,3.0
2,2013-01-01 02:00:00,1.5,2.0,71.3,3.3,2.0
3,2013-01-01 03:00:00,1.6,2.0,70.7,2.9,2.0
4,2013-01-01 04:00:00,1.5,2.0,70.4,3.1,2.0
...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,NaN,2.0,103.9,0.9,2.0
8756,2013-12-31 20:00:00,NaN,2.0,104.4,0.8,2.0
8757,2013-12-31 21:00:00,NaN,4.0,105.3,0.7,1.0
8758,2013-12-31 22:00:00,NaN,4.0,106.4,1.0,1.0


## 2014

In [27]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2014")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [28]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2014/"+file
    #read to a pandas dataframe
    df_2014 = pd.read_excel(xls)

In [29]:
# Let´s rename the column "Fail" to "Date"
df_2014.rename(columns={"Faial":"Date"}, inplace=True)
df_2014

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2014-01-01 00:00:00,NaN,3.0,111.7,0.8,2.0
1,2014-01-01 01:00:00,NaN,3.0,107.2,0.9,2.0
2,2014-01-01 02:00:00,NaN,3.0,105.6,0.9,2.0
3,2014-01-01 03:00:00,NaN,8.7,106.0,0.8,2.0
4,2014-01-01 04:00:00,NaN,9.0,105.2,0.8,2.0
...,...,...,...,...,...,...
8755,2014-12-31 19:00:00,3.2,1.7,75.3,0.2,0.1
8756,2014-12-31 20:00:00,3.1,2.1,NaN,0.3,0.3
8757,2014-12-31 21:00:00,3.1,2.3,76.5,0.3,0.4
8758,2014-12-31 22:00:00,3.1,2.3,77.5,0.1,0.4


## 2015

In [30]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2015")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [31]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2015/"+file
    #read to a pandas dataframe
    df_2015 = pd.read_excel(xls)

In [32]:
df_2015

,Faial,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2015-01-01 00:00:00,3.3,2.4,76.9,0.3,0.2
1,2015-01-01 01:00:00,3.2,2.4,75.9,0.3,0.2
2,2015-01-01 02:00:00,3.4,2.7,77.1,0.5,0.1
3,2015-01-01 03:00:00,3.0,2.8,78.6,0.4,0.1
4,2015-01-01 04:00:00,2.8,2.8,78.5,0.1,0.1
...,...,...,...,...,...,...
8755,2015-12-31 19:00:00,6.4,11.4,86.6,0.4,4.6
8756,2015-12-31 20:00:00,6.7,14.5,85.2,0.5,6.9
8757,2015-12-31 21:00:00,6.4,14.5,83.4,0.5,6.9
8758,2015-12-31 22:00:00,6.0,15.8,84.6,0.5,6.7


In [33]:
# Let´s rename the column "Fail" to "Date"
df_2015.rename(columns={"Faial":"Date"}, inplace=True)

In [34]:
df_2015

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2015-01-01 00:00:00,3.3,2.4,76.9,0.3,0.2
1,2015-01-01 01:00:00,3.2,2.4,75.9,0.3,0.2
2,2015-01-01 02:00:00,3.4,2.7,77.1,0.5,0.1
3,2015-01-01 03:00:00,3.0,2.8,78.6,0.4,0.1
4,2015-01-01 04:00:00,2.8,2.8,78.5,0.1,0.1
...,...,...,...,...,...,...
8755,2015-12-31 19:00:00,6.4,11.4,86.6,0.4,4.6
8756,2015-12-31 20:00:00,6.7,14.5,85.2,0.5,6.9
8757,2015-12-31 21:00:00,6.4,14.5,83.4,0.5,6.9
8758,2015-12-31 22:00:00,6.0,15.8,84.6,0.5,6.7


## 2016

In [35]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2016")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [36]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2016/"+file
    #read to a pandas dataframe
    df_2016 = pd.read_excel(xls)

In [37]:
# Let´s rename the column "Fail" to "Date"
df_2016.rename(columns={"Faial":"Date"}, inplace=True)
df_2016

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2016-01-01 00:00:00,5.3,14.2,82.8,0.8,NaN
1,2016-01-01 01:00:00,5.1,11.0,81.9,0.4,NaN
2,2016-01-01 02:00:00,4.8,11.0,81.4,0.8,NaN
3,2016-01-01 03:00:00,5.0,9.2,82.6,1.1,NaN
4,2016-01-01 04:00:00,4.6,7.3,82.1,0.3,NaN
...,...,...,...,...,...,...
8779,2016-12-31 19:00:00,6.9,1.2,92.7,0.2,1.4
8780,2016-12-31 20:00:00,6.6,1.2,94.5,0.0,1.4
8781,2016-12-31 21:00:00,5.8,2.4,95.3,0.2,1.4
8782,2016-12-31 22:00:00,5.6,5.9,95.1,0.4,1.4


## 2017

In [38]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2017")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [40]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2017/"+file
    #read to a pandas dataframe
    df_2017= pd.read_excel(xls)

In [41]:
# Let´s rename the column "Fail" to "Date"
df_2017.rename(columns={"Faial":"Date"}, inplace=True)
df_2017

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2017-01-01 00:00:00,5.5,1.3,95.8,0.2,5.1
1,2017-01-01 01:00:00,5.2,0.9,96.0,0.5,3.1
2,2017-01-01 02:00:00,5.2,0.9,95.9,0.2,3.1
3,2017-01-01 03:00:00,3.7,2.5,95.7,0.1,2.7
4,2017-01-01 04:00:00,1.3,6.9,95.4,0.3,1.7
...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,0.0,11.4,86.5,3.1,3.3
8756,2017-12-31 20:00:00,0.0,10.0,86.3,2.2,3.0
8757,2017-12-31 21:00:00,0.0,10.0,85.9,2.3,3.0
8758,2017-12-31 22:00:00,0.0,7.4,84.9,1.5,2.9


## 2018

In [42]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2018")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [43]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2018/"+file
    #read to a pandas dataframe
    df_2018= pd.read_excel(xls)

In [44]:
df_2018

,Faial,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2018-01-01 00:00:00,1.9,5.2,83.3,2.2,2.9
1,2018-01-01 01:00:00,2.3,15.5,84.0,2.3,2.6
2,2018-01-01 02:00:00,1.9,24.5,81.9,2.5,2.4
3,2018-01-01 03:00:00,1.8,24.5,80.2,2.2,2.4
4,2018-01-01 04:00:00,1.9,18.4,77.1,1.2,NaN
...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,0.0,3.3,78.4,1.5,2.8
8756,2018-12-31 20:00:00,0.3,4.3,85.0,1.3,3.2
8757,2018-12-31 21:00:00,0.5,5.3,75.6,2.9,4.4
8758,2018-12-31 22:00:00,0.5,5.3,81.1,1.3,4.4


In [45]:
# Let´s rename the column "Fail" to "Date"
df_2018.rename(columns={"Faial":"Date"}, inplace=True)
df_2018

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2018-01-01 00:00:00,1.9,5.2,83.3,2.2,2.9
1,2018-01-01 01:00:00,2.3,15.5,84.0,2.3,2.6
2,2018-01-01 02:00:00,1.9,24.5,81.9,2.5,2.4
3,2018-01-01 03:00:00,1.8,24.5,80.2,2.2,2.4
4,2018-01-01 04:00:00,1.9,18.4,77.1,1.2,NaN
...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,0.0,3.3,78.4,1.5,2.8
8756,2018-12-31 20:00:00,0.3,4.3,85.0,1.3,3.2
8757,2018-12-31 21:00:00,0.5,5.3,75.6,2.9,4.4
8758,2018-12-31 22:00:00,0.5,5.3,81.1,1.3,4.4


In [46]:
df_2018

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2018-01-01 00:00:00,1.9,5.2,83.3,2.2,2.9
1,2018-01-01 01:00:00,2.3,15.5,84.0,2.3,2.6
2,2018-01-01 02:00:00,1.9,24.5,81.9,2.5,2.4
3,2018-01-01 03:00:00,1.8,24.5,80.2,2.2,2.4
4,2018-01-01 04:00:00,1.9,18.4,77.1,1.2,NaN
...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,0.0,3.3,78.4,1.5,2.8
8756,2018-12-31 20:00:00,0.3,4.3,85.0,1.3,3.2
8757,2018-12-31 21:00:00,0.5,5.3,75.6,2.9,4.4
8758,2018-12-31 22:00:00,0.5,5.3,81.1,1.3,4.4


## 2019

In [47]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2019")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [48]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2019/"+file
    #read to a pandas dataframe
    df_2019 = pd.read_excel(xls)

In [49]:
# Let´s rename the column "Fail" to "Date"
df_2019.rename(columns={"Faial":"Date"}, inplace=True)
df_2019

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2019-01-01 00:00:00,1.1,4.3,79.4,1.5,4.1
1,2019-01-01 01:00:00,1.1,4.3,79.6,2.5,4.1
2,2019-01-01 02:00:00,1.1,4.4,83.3,1.5,NaN
3,2019-01-01 03:00:00,1.4,4.5,83.9,1.2,NaN
4,2019-01-01 04:00:00,1.5,4.5,83.1,1.7,NaN
...,...,...,...,...,...,...
8755,2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN
8756,2019-12-31 20:00:00,0.0,11.2,81.6,2.6,9.6
8757,2019-12-31 21:00:00,0.0,11.2,78.6,4.2,NaN
8758,2019-12-31 22:00:00,0.0,10.5,79.6,3.5,NaN


## 2020

In [50]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2020")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [51]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2020/"+file
    #read to a pandas dataframe
    df_2020 = pd.read_excel(xls)

In [52]:
# Let´s rename the column "Fail" to "Date"
df_2020.rename(columns={"Faial":"Date"}, inplace=True)
df_2020

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2020-01-01 00:00:00,0.0,8.5,79.5,4.5,NaN
1,2020-01-01 01:00:00,0.0,7.9,82.3,3.7,NaN
2,2020-01-01 02:00:00,0.0,6.2,87.3,2.0,NaN
3,2020-01-01 03:00:00,0.5,6.2,90.6,2.0,NaN
4,2020-01-01 04:00:00,2.1,6.1,92.2,1.8,NaN
...,...,...,...,...,...,...
8779,2020-12-31 19:00:00,1.7,6.5,NaN,1.3,2.4
8780,2020-12-31 20:00:00,1.8,7.1,NaN,1.5,2.1
8781,2020-12-31 21:00:00,1.9,7.1,NaN,2.2,2.1
8782,2020-12-31 22:00:00,2.0,7.1,NaN,2.3,2.1


## 2021

In [53]:
# Let´s list the files in the data directory
files = os.listdir("./qualar-data-2021")
print(files)

['station_1021.xlsx', 'station_1023.xlsx', 'station_1024.xlsx', 'station_1025.xlsx', 'station_1028.xlsx', 'station_1030.xlsx', 'station_1031.xlsx', 'station_1034.xlsx', 'station_1041.xlsx', 'station_1042.xlsx', 'station_1043.xlsx', 'station_1044.xlsx', 'station_1046.xlsx', 'station_1047.xlsx', 'station_1048.xlsx', 'station_1050.xlsx', 'station_1051.xlsx', 'station_1052.xlsx', 'station_1053.xlsx', 'station_1054.xlsx', 'station_1055.xlsx', 'station_1100.xlsx', 'station_2004.xlsx', 'station_2006.xlsx', 'station_2016.xlsx', 'station_2017.xlsx', 'station_2018.xlsx', 'station_2019.xlsx', 'station_2020.xlsx', 'station_2021.xlsx', 'station_2022.xlsx', 'station_3055.xlsx', 'station_3063.xlsx', 'station_3070.xlsx', 'station_3071.xlsx', 'station_3072.xlsx', 'station_3075.xlsx', 'station_3082.xlsx', 'station_3083.xlsx', 'station_3084.xlsx', 'station_3085.xlsx', 'station_3087.xlsx', 'station_3089.xlsx', 'station_3091.xlsx', 'station_3092.xlsx', 'station_3093.xlsx', 'station_3094.xlsx', 'station_309

In [54]:
# For each file, let´s import to a pandas dataframe
for file in files:
    #define the file name
    xls = "./qualar-data-2021/"+file
    #read to a pandas dataframe
    df_2021 = pd.read_excel(xls)

In [55]:
# Let´s rename the column "Fail" to "Date"
df_2021.rename(columns={"Faial":"Date"}, inplace=True)
df_2021

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2021-01-01 00:00:00,2.1,6.8,NaN,1.9,2.2
1,2021-01-01 01:00:00,2.1,6.8,NaN,1.5,2.2
2,2021-01-01 02:00:00,2.4,7.9,NaN,3.0,1.6
3,2021-01-01 03:00:00,1.9,7.9,NaN,2.5,1.6
4,2021-01-01 04:00:00,1.6,7.9,NaN,2.1,1.6
...,...,...,...,...,...,...
8755,2021-12-31 19:00:00,2.3,7.9,82.9,1.7,6.6
8756,2021-12-31 20:00:00,5.8,7.9,78.4,1.4,6.6
8757,2021-12-31 21:00:00,5.9,7.8,83.2,1.8,5.8
8758,2021-12-31 22:00:00,6.0,7.6,82.1,1.8,5.0


# Let´s Concat all The Data set´s in One Singel Data Set

In [56]:
df_2011

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2011-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN
2,2011-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN
3,2011-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN
4,2011-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8755,2011-12-31 19:00:00,1.3,3.2,70.6,1.1,3.0
8756,2011-12-31 20:00:00,1.3,4.0,72.6,1.0,2.7
8757,2011-12-31 21:00:00,1.2,4.0,71.8,1.0,2.0
8758,2011-12-31 22:00:00,1.4,3.7,66.2,1.1,2.0


In [57]:
df_2012

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2012-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2012-01-01 01:00:00,1.4,3.0,69.9,1.1,1.0
2,2012-01-01 02:00:00,1.2,3.0,73.0,1.1,1.5
3,2012-01-01 03:00:00,1.2,3.0,72.6,1.3,2.0
4,2012-01-01 04:00:00,1.5,3.0,74.9,1.2,2.0
...,...,...,...,...,...,...
8779,2012-12-31 19:00:00,1.3,1.2,75.5,1.7,2.0
8780,2012-12-31 20:00:00,1.5,4.0,71.3,1.6,5.0
8781,2012-12-31 21:00:00,1.5,4.0,69.9,1.6,5.0
8782,2012-12-31 22:00:00,1.4,4.0,70.0,1.6,4.7


In [58]:
df_2013

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2013-01-01 00:00:00,1.5,3.0,72.4,3.1,3.0
1,2013-01-01 01:00:00,1.4,2.7,71.9,4.1,3.0
2,2013-01-01 02:00:00,1.5,2.0,71.3,3.3,2.0
3,2013-01-01 03:00:00,1.6,2.0,70.7,2.9,2.0
4,2013-01-01 04:00:00,1.5,2.0,70.4,3.1,2.0
...,...,...,...,...,...,...
8755,2013-12-31 19:00:00,NaN,2.0,103.9,0.9,2.0
8756,2013-12-31 20:00:00,NaN,2.0,104.4,0.8,2.0
8757,2013-12-31 21:00:00,NaN,4.0,105.3,0.7,1.0
8758,2013-12-31 22:00:00,NaN,4.0,106.4,1.0,1.0


In [59]:
df_2014

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2014-01-01 00:00:00,NaN,3.0,111.7,0.8,2.0
1,2014-01-01 01:00:00,NaN,3.0,107.2,0.9,2.0
2,2014-01-01 02:00:00,NaN,3.0,105.6,0.9,2.0
3,2014-01-01 03:00:00,NaN,8.7,106.0,0.8,2.0
4,2014-01-01 04:00:00,NaN,9.0,105.2,0.8,2.0
...,...,...,...,...,...,...
8755,2014-12-31 19:00:00,3.2,1.7,75.3,0.2,0.1
8756,2014-12-31 20:00:00,3.1,2.1,NaN,0.3,0.3
8757,2014-12-31 21:00:00,3.1,2.3,76.5,0.3,0.4
8758,2014-12-31 22:00:00,3.1,2.3,77.5,0.1,0.4


In [60]:
df_2015

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2015-01-01 00:00:00,3.3,2.4,76.9,0.3,0.2
1,2015-01-01 01:00:00,3.2,2.4,75.9,0.3,0.2
2,2015-01-01 02:00:00,3.4,2.7,77.1,0.5,0.1
3,2015-01-01 03:00:00,3.0,2.8,78.6,0.4,0.1
4,2015-01-01 04:00:00,2.8,2.8,78.5,0.1,0.1
...,...,...,...,...,...,...
8755,2015-12-31 19:00:00,6.4,11.4,86.6,0.4,4.6
8756,2015-12-31 20:00:00,6.7,14.5,85.2,0.5,6.9
8757,2015-12-31 21:00:00,6.4,14.5,83.4,0.5,6.9
8758,2015-12-31 22:00:00,6.0,15.8,84.6,0.5,6.7


In [61]:
df_2016

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2016-01-01 00:00:00,5.3,14.2,82.8,0.8,NaN
1,2016-01-01 01:00:00,5.1,11.0,81.9,0.4,NaN
2,2016-01-01 02:00:00,4.8,11.0,81.4,0.8,NaN
3,2016-01-01 03:00:00,5.0,9.2,82.6,1.1,NaN
4,2016-01-01 04:00:00,4.6,7.3,82.1,0.3,NaN
...,...,...,...,...,...,...
8779,2016-12-31 19:00:00,6.9,1.2,92.7,0.2,1.4
8780,2016-12-31 20:00:00,6.6,1.2,94.5,0.0,1.4
8781,2016-12-31 21:00:00,5.8,2.4,95.3,0.2,1.4
8782,2016-12-31 22:00:00,5.6,5.9,95.1,0.4,1.4


In [62]:
df_2017

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2017-01-01 00:00:00,5.5,1.3,95.8,0.2,5.1
1,2017-01-01 01:00:00,5.2,0.9,96.0,0.5,3.1
2,2017-01-01 02:00:00,5.2,0.9,95.9,0.2,3.1
3,2017-01-01 03:00:00,3.7,2.5,95.7,0.1,2.7
4,2017-01-01 04:00:00,1.3,6.9,95.4,0.3,1.7
...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,0.0,11.4,86.5,3.1,3.3
8756,2017-12-31 20:00:00,0.0,10.0,86.3,2.2,3.0
8757,2017-12-31 21:00:00,0.0,10.0,85.9,2.3,3.0
8758,2017-12-31 22:00:00,0.0,7.4,84.9,1.5,2.9


In [63]:
df_2018

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2018-01-01 00:00:00,1.9,5.2,83.3,2.2,2.9
1,2018-01-01 01:00:00,2.3,15.5,84.0,2.3,2.6
2,2018-01-01 02:00:00,1.9,24.5,81.9,2.5,2.4
3,2018-01-01 03:00:00,1.8,24.5,80.2,2.2,2.4
4,2018-01-01 04:00:00,1.9,18.4,77.1,1.2,NaN
...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,0.0,3.3,78.4,1.5,2.8
8756,2018-12-31 20:00:00,0.3,4.3,85.0,1.3,3.2
8757,2018-12-31 21:00:00,0.5,5.3,75.6,2.9,4.4
8758,2018-12-31 22:00:00,0.5,5.3,81.1,1.3,4.4


In [64]:
df_2019

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2019-01-01 00:00:00,1.1,4.3,79.4,1.5,4.1
1,2019-01-01 01:00:00,1.1,4.3,79.6,2.5,4.1
2,2019-01-01 02:00:00,1.1,4.4,83.3,1.5,NaN
3,2019-01-01 03:00:00,1.4,4.5,83.9,1.2,NaN
4,2019-01-01 04:00:00,1.5,4.5,83.1,1.7,NaN
...,...,...,...,...,...,...
8755,2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN
8756,2019-12-31 20:00:00,0.0,11.2,81.6,2.6,9.6
8757,2019-12-31 21:00:00,0.0,11.2,78.6,4.2,NaN
8758,2019-12-31 22:00:00,0.0,10.5,79.6,3.5,NaN


In [65]:
df_2020

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2020-01-01 00:00:00,0.0,8.5,79.5,4.5,NaN
1,2020-01-01 01:00:00,0.0,7.9,82.3,3.7,NaN
2,2020-01-01 02:00:00,0.0,6.2,87.3,2.0,NaN
3,2020-01-01 03:00:00,0.5,6.2,90.6,2.0,NaN
4,2020-01-01 04:00:00,2.1,6.1,92.2,1.8,NaN
...,...,...,...,...,...,...
8779,2020-12-31 19:00:00,1.7,6.5,NaN,1.3,2.4
8780,2020-12-31 20:00:00,1.8,7.1,NaN,1.5,2.1
8781,2020-12-31 21:00:00,1.9,7.1,NaN,2.2,2.1
8782,2020-12-31 22:00:00,2.0,7.1,NaN,2.3,2.1


In [66]:
df_2021

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2021-01-01 00:00:00,2.1,6.8,NaN,1.9,2.2
1,2021-01-01 01:00:00,2.1,6.8,NaN,1.5,2.2
2,2021-01-01 02:00:00,2.4,7.9,NaN,3.0,1.6
3,2021-01-01 03:00:00,1.9,7.9,NaN,2.5,1.6
4,2021-01-01 04:00:00,1.6,7.9,NaN,2.1,1.6
...,...,...,...,...,...,...
8755,2021-12-31 19:00:00,2.3,7.9,82.9,1.7,6.6
8756,2021-12-31 20:00:00,5.8,7.9,78.4,1.4,6.6
8757,2021-12-31 21:00:00,5.9,7.8,83.2,1.8,5.8
8758,2021-12-31 22:00:00,6.0,7.6,82.1,1.8,5.0


In [67]:
df = pd.concat([df_2011,df_2012,df_2013,df_2014,df_2015, df_2016,df_2017,df_2018,df_2019,df_2020,df_2021], axis=0)

In [68]:
df

,Date,Dióxido de Enxofre (µg/m3),Partículas < 10 µm (µg/m3),Ozono (µg/m3),Dióxido de Azoto (µg/m3),Partículas < 2.5 µm (µg/m3)
0,2011-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2011-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN
2,2011-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN
3,2011-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN
4,2011-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
8755,2021-12-31 19:00:00,2.3,7.9,82.9,1.7,6.6
8756,2021-12-31 20:00:00,5.8,7.9,78.4,1.4,6.6
8757,2021-12-31 21:00:00,5.9,7.8,83.2,1.8,5.8
8758,2021-12-31 22:00:00,6.0,7.6,82.1,1.8,5.0


In [69]:
df.to_excel("air_quality.xlsx")